In [ ]:
import netCDF4
from netCDF4 import num2date
import numpy as np
import os
import pandas as pd

In [ ]:
df = netCDF4.Dataset(r"data\cdf\gpcp_v02r03_monthly_d202401_c20240407.nc")
df.variables

In [ ]:
t2m = df.variables["precip"]

In [ ]:
time_dim, lat_dim, lon_dim = t2m.get_dims()
time_var = df.variables[time_dim.name]
times = num2date(time_var[:], time_var.units)
latitudes = df.variables[lat_dim.name][:]
longitudes = df.variables[lon_dim.name][:]

In [ ]:
output_dir = "./"

# =============================== METHOD 1 ============================
# Extract each time as a 2D pandas DataFrame and write it to CSV
# =====================================================================

In [ ]:
filename = os.path.join(output_dir, "table.csv")
print(f"Writing data in tabular form to {filename} (this may take some time)...")
times_grid, latitudes_grid, longitudes_grid = [
    x.flatten() for x in np.meshgrid(times, latitudes, longitudes, indexing="ij")
]
df = pd.DataFrame(
    {
        "time": [t.isoformat() for t in times_grid],
        "latitude": latitudes_grid,
        "longitude": longitudes_grid,
        "t2m": t2m[:].flatten(),
    }
)
df.to_csv(filename, index=False)
print("Done")

# =============================== METHOD 2 ============================
# Write data as a table with 4 columns: time, latitude, longitude, value
# =====================================================================

In [ ]:
os.makedirs(output_dir, exist_ok=True)
for i, t in enumerate(times):
    filename = os.path.join(output_dir, f"{t.isoformat()}.csv")
    print(f"Writing time {t} to {filename}")
    df = pd.DataFrame(t2m[i, :, :], index=latitudes, columns=longitudes)
    df.to_csv(filename)
print("Done")